### Getting Ready

We'll start by importing the packages we need.  In addition to the diffusers library, we'll load two new modules from Hugging Face.
- [datasets](https://huggingface.co/docs/datasets/index) gives access to data sets stored on [Hugging Face Hub](https://huggingface.co/datasets).
- [huggingface_hub](https://huggingface.co/docs/huggingface_hub/index) is a Python client to the Hugging Face Hub API.

In [ ]:
import os
import sys

import datasets
import diffusers
import huggingface_hub
import requests
import torch
from dotenv import load_dotenv
from huggingface_hub import HfApi
from IPython.display import display

We'll print out version number of the critical packages, to help with future reproducibility.

In [ ]:
print("Platform:", sys.platform)
print("Python version:", sys.version)
print("---")
print("datasets version: ", datasets.__version__)
print("diffusers version: ", diffusers.__version__)
print("huggingface_hub version: ", huggingface_hub.__version__)
print("torch version:", torch.__version__)

Let's check if a GPU is available.  If not, this notebook will take a long time to run!

In [ ]:
if torch.cuda.is_available():
    device = "cuda"
    dtype = torch.float16
else:
    device = "cpu"
    dtype = torch.float32

print(f"Using {device} device with {dtype} data type.")

### Load Stable Diffusion

We'll start by loading in the same Stable Diffusion pipeline we used in the previous lesson.  We could use the `StableDiffusionPipeline` as before, if we wanted.  But the diffusers library has a handy `AutoPipelineForText2Image` class, which will figure out the right pipeline to use based on the model it's given.

**Task 6.3.1:** Load the Stable Diffusion v1.4 pipeline.

In [ ]:
MODEL_NAME = "CompVis/stable-diffusion-v1-4"

pipeline = ...
pipeline.to(device)

print(type(pipeline))

Our goal is to use this to generate images of our dog Maya.  Here's a picture:

![Maya](images/maya.jpg)

Does Stable Diffusion already know what she looks like?

**Task 6.3.2:** Generate four images with the prompt, "My dog Maya."

In [ ]:
images = ...

for im in images:
    display(im)

### Load Data Set for Fine Tuning

While Stable Diffusion can produce many cute looking dogs, it obviously doesn't know what Maya looks like.  And that's expected&mdash;clearly the model can't know what every dog in the entire world looks like!

We can fix that by fine-tuning the model on pictures of Maya.  To do this, we'll need a data set with pictures of Maya.  Luckily for us, this [data set already exists](https://huggingface.co/datasets/worldquant-university/maya-dataset-v1) on Hugging Face Hub!

Data sets on Hugging Face Hub are identified as `user/dataset`.

**Task 6.3.3:** Create a data set builder, which will let us access metadata about this data set.

In [ ]:
DATASET_NAME = ...
data_builder = datasets.load_dataset_builder(DATASET_NAME)

print(data_builder.dataset_name)

This builder object has downloaded metadata about the data set.  This allows us to see what features are available, for example.

In [ ]:
print(data_builder.info.features)

**Task 6.3.4:** Determine which splits are included.  There is a `.splits` attribute in the `.info` property of our builder.

In [ ]:
print(...)

This builder didn't download all of the data.  This is nice in general, since many data sets can be very large.  We can see above that there are only 26 rows in this dataset, so we can afford to download the whole thing.

**Task 6.3.5:** Load the Maya data set from Hugging Face Hub.

In [ ]:
data = datasets.load_dataset(...)

print(data)

This `DatasetDict` acts like a dictionary.  We can use key indexing to find the data itself.

In [ ]:
data["train"]["image"]

**Task 6.3.6:** Display the fourth image. (Or take a look at more of them, if you like!)

In [ ]:
# The values are PIL images, so they will be displayed
# automatically by Jupyter.


**Task 6.3.7:** Print out the text associated with each row in the training data.

In [ ]:
# Use dictionary indexing to look up the text values.


If the fine-tuning works, the model should figure out what "My dog Maya" looks like.

### LoRA Fine-tuning

We have the model and we have the data, so we're ready to do the LoRA fine-tuning.  The actual process is rather involved.  Luckily, we don't have to program it ourselves.  The diffusers project includes [a script](https://github.com/huggingface/diffusers/blob/main/examples/text_to_image/train_text_to_image_lora.py) that does LoRA fine-tuning of a Hugging Face model based on a Hugging Face data set.  There's a copy of this script in the same directory as this notebook: [`train_text_to_image_lora.py`](train_text_to_image_lora.py).

In addition to the model and data set names, we'll need to tell the script where to save the LoRA weights.  It's up to you, but we'd recommend `maya_model_v1_lora`.

**Task 6.3.8:** Set the `OUTPUT_DIR` variable with the correct value.

In [ ]:
OUTPUT_DIR = ...

**Task 6.3.9:** Inspect the `train_text_to_image_lora.py` script.

**Due to GPU limitations you CAN NOT run this script.**

It's an advanced (and long) script. But if you want, take a look at it to familiarize yourself with the process of fine tuning.

<div class="alert alert-info" role="alert">
    <p><b>Changes with respect to the video</b></p>
    <p>The instructor in the video is executing the script in the notebook. But due to high GPU usage, we can't replicate it in your interactive environment.</p>
    <p><b>Please, do NOT try running the script as it'll fail and restart your lab environment</b></p>
    <p>Instead, we have already ran the script for you with different training epochs and stored the results in the output directory.</p>
</div>

This command could take an hour to complete.  Once it's done, there should be a `pytorch_lora_weights.safetensors` file in the output directory.

In [ ]:
!ls -lah $OUTPUT_DIR

<div class="alert alert-info" role="alert">
    <p><b>Changes with respect to the video</b></p>
    <p>We have provided MULTIPLE examples of fine tuning with different epochs so you can compare.</p>
</div>

### Load LoRA Weights

Now that we have the LoRA weights, we can load them into the pipeline we already have.  The pipeline has a `.load_lora_weights()` method that can take a directory path as an argument.  The file name of the weights file is given separately in the `weight_name` argument.

**Task 6.3.10:** Load the LoRA weights into our pipeline.

<div class="alert alert-info" role="alert">
    <p><b>Changes with respect to the video</b></p>
    <p>We have picked the weights trained for 10 epochs. You can choose your own. If you're going to choose another one, make sure to invoke:
    <code><pre>pipeline.unload_lora_weights()</pre></code></p>
</div>

In [ ]:
pipeline.load_lora_weights(
    ..., # Directory containing weights file

    weight_name="pytorch_lora_weights.10_epochs.safetensors",
)

**Task 6.3.11:** Generate four images of "My dog Maya" with the fine-tuned model.

In [ ]:
images = ...

for im in images:
    display(im)

These should look (a bit) more like Maya!

# Conclusion

You've now used the fine-tuned model to produce images of our dog Maya that look (a bit more) like her.  If we had more training data and had trained for more epochs, perhaps the model would perform even better.  By using the LoRA algorithm, we could do the fine-tuning with only a fraction of the parameters in the original model.  (Note that the file containing the LoRA weights is about 3 MB in size.  The full weights for the Stable Diffusion model are about 3 GB in size!)

In the process, we've learned:
- How to download data sets from Hugging Face.
- How to run LoRA from a script.
- How to load the LoRA weights into a Stable Diffusion pipeline.
- How to set up a Hugging Face token for API access.
- How to upload weights to Hugging Face repository.

In the next lesson, we'll use our fine-tuned model as a component in a Streamlit web app.